In [1]:
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import glob
import os
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import csv
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string
import itertools
import zipfile
import json
punctuation = string.punctuation
stopwordsset = set(stopwords.words("english"))
stopwordsset.add("'s")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alch5988/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alch5988/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alch5988/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Removing urls
def removeURL(text):
    result = re.sub(r"http\S+", "", text)
    return result
#Extracting contextual words from a sentence
def tokenize(text):
    #lower case
    text = text.lower()
    #split into individual words
    words = word_tokenize(text)
    return words
def stem(tokenizedtext):
    rootwords = []
    for aword in tokenizedtext:
        aword = ps.stem(aword)
        rootwords.append(aword)
    return rootwords
def stopWords(tokenizedtext):
    goodwords = []
    for aword in tokenizedtext:
        if aword not in stopwordsset:
            goodwords.append(aword)
    return goodwords
def lemmatizer(tokenizedtext):
    lemmawords = []
    for aword in tokenizedtext:
        aword = wn.lemmatize(aword)
        lemmawords.append(aword)
    return lemmawords
def removePunctuation(tokenizedtext):
    nopunctwords = []
    for aword in tokenizedtext:
        if aword not in punctuation:
            nopunctwords.append(aword)
    cleanedwords = []
    for aword in nopunctwords:
        aword = aword.translate(str.maketrans('', '', string.punctuation))
        cleanedwords.append(aword)
    return cleanedwords

In [3]:
TMPDIR = 'tmp'

In [4]:
if not os.path.exists(TMPDIR):
    os.makedirs(TMPDIR)

In [5]:
tweetzipfiles = glob.glob('*zip')

In [6]:
for tweetzipfile in tweetzipfiles:
    with zipfile.ZipFile(tweetzipfile, "r") as f:
        print("Unzipping to tmp directory %s" % tweetzipfile)
        f.extractall(TMPDIR)

Unzipping to tmp directory hbo.zip
Unzipping to tmp directory hulu.zip
Unzipping to tmp directory netflix.zip


In [7]:
#for each file in directory
uniquewords = {}
count = 0 

for fn in os.listdir(TMPDIR):
    fn = os.path.join(TMPDIR, fn)
    with open(fn) as f:
        tweetjson = json.load(f)
        count += 1
        if count % 1000 == 0:
            print(count)
            
        text = tweetjson['text']
        nourlstext = removeURL(text)
        tokenizedtext = tokenize(nourlstext)
        nostopwordstext = stopWords(tokenizedtext)
        lemmatizedtext = lemmatizer(nostopwordstext)
        nopuncttext = removePunctuation(lemmatizedtext)
        
        for aword in nopuncttext:
            if aword in uniquewords:
                uniquewords[aword] += 1
            if aword not in uniquewords:
                uniquewords[aword] = 1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [11]:
wordstoinclude = set()
wordcount = 0
for aword in uniquewords:
    if uniquewords[aword] > 50: #mentioned more than 50 times.. we used baseline of 25 in class but need to figure out optimal number for my network specifically
        wordcount += 1
        wordstoinclude.add(aword)

In [12]:
print(wordcount)

973


In [15]:
edgelist = open('semantic.edgelist.for.gephi.csv', 'w')
csvwriter = csv.writer(edgelist)

#header for Gephi to specify there are undirected ties between words
header = ['Source', 'Target', "Type"]
csvwriter.writerow(header)

print('Writing Edge List')

uniquewords = {}
count = 0 

for fn in os.listdir(TMPDIR):
    fn = os.path.join(TMPDIR, fn)
    with open(fn) as f:
        tweetjson = json.load(f)
        count += 1
        if count % 1000 == 0:
            print(count)
            
        text = tweetjson['text']
        nourlstext = removeURL(text)
        tokenizedtext = tokenize(nourlstext)
        nostopwordstext = stopWords(tokenizedtext)
        lemmatizedtext = lemmatizer(nostopwordstext)
        nopuncttext = removePunctuation(lemmatizedtext)
        
        goodwords = []
        for aword in nopuncttext:
            if aword in wordstoinclude:
                goodwords.append(aword.replace(',',''))
        
        allcombos = itertools.combinations(goodwords, 2)
        for acombo in allcombos:
            row = []
            for anode in acombo:
                row.append(anode)
            row.append('Undirected')
            csvwriter.writerow(row)
edgelist.close()

Writing Edge List
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [17]:
#Shut down temp directory
shutil.rmtree(TMPDIR)